In [1]:
import random
import json
import pickle
import nltk
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')

Using TensorFlow backend.
[nltk_data] Downloading package punkt to /home/ayon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ayon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
LU_words=[]
tags = []
pair = []
ignore_words = ['?', '!', '.']
LU_file = open('Luluintents.json').read()
LU_intents = json.loads(LU_file)

In [3]:
for intent in LU_intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        LU_words.extend(w)
        pair.append((w, intent['tag']))
        if intent['tag'] not in tags:
            tags.append(intent['tag'])

In [7]:
LU_words = [lemmatizer.lemmatize(w.lower()) for w in LU_words if w not in ignore_words]
LU_words = sorted(list(set(LU_words)))
tags = sorted(list(set(tags)))
print (len(pair), "Pairs of Patterns")
print (len(tags), "Unique Tags", tags)
print (len(LU_words), "Unique Lemmatized Words", LU_words)

27 Pairs of Patterns
5 Unique Tags ['goodbye', 'greeting', 'kyc_form', 'options', 'thanks']
56 Unique Lemmatized Words ["'s", ',', 'a', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'fill', 'find', 'for', 'form', 'get', 'give', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'kyc', 'later', 'me', 'new', 'next', 'nice', 'offered', 'provide', 'registration', 'see', 'should', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'user', 'what', 'where', 'which', 'you']


In [8]:
pickle.dump(LU_words,open('words.pkl','wb'))
pickle.dump(tags,open('classes.pkl','wb'))

In [13]:
training = []
output_empty = [0] * len(tags)
for doc in pair:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in LU_words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[tags.index(doc[1])] = 1
    training.append([bag, output_row])

In [14]:
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

Training data created


In [15]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("NLP Model Created")

Epoch 1/200
27/27 [==============================] - 1s 26ms/step - loss: 1.6221 - accuracy: 0.1852
Epoch 2/200
27/27 [==============================] - 0s 601us/step - loss: 1.5862 - accuracy: 0.2963
Epoch 3/200
27/27 [==============================] - 0s 496us/step - loss: 1.5152 - accuracy: 0.2963
Epoch 4/200
27/27 [==============================] - 0s 546us/step - loss: 1.4363 - accuracy: 0.3333
Epoch 5/200
27/27 [==============================] - 0s 642us/step - loss: 1.3526 - accuracy: 0.6296
Epoch 6/200
27/27 [==============================] - 0s 560us/step - loss: 1.3410 - accuracy: 0.4074
Epoch 7/200
27/27 [==============================] - 0s 510us/step - loss: 1.2715 - accuracy: 0.7407
Epoch 8/200
27/27 [==============================] - 0s 534us/step - loss: 1.1534 - accuracy: 0.6667
Epoch 9/200
27/27 [==============================] - 0s 556us/step - loss: 1.0613 - accuracy: 0.8148
Epoch 10/200
27/27 [==============================] - 0s 523us/step - loss: 1.0597 - accurac

27/27 [==============================] - 0s 421us/step - loss: 0.0135 - accuracy: 1.0000
Epoch 82/200
27/27 [==============================] - 0s 674us/step - loss: 0.0105 - accuracy: 1.0000
Epoch 83/200
27/27 [==============================] - 0s 702us/step - loss: 0.0229 - accuracy: 1.0000
Epoch 84/200
27/27 [==============================] - 0s 492us/step - loss: 0.0254 - accuracy: 1.0000
Epoch 85/200
27/27 [==============================] - 0s 395us/step - loss: 0.0408 - accuracy: 1.0000
Epoch 86/200
27/27 [==============================] - 0s 318us/step - loss: 0.0245 - accuracy: 1.0000
Epoch 87/200
27/27 [==============================] - 0s 380us/step - loss: 0.0421 - accuracy: 1.0000
Epoch 88/200
27/27 [==============================] - 0s 511us/step - loss: 0.0352 - accuracy: 1.0000
Epoch 89/200
27/27 [==============================] - 0s 462us/step - loss: 0.0191 - accuracy: 1.0000
Epoch 90/200
27/27 [==============================] - 0s 375us/step - loss: 0.0435 - accuracy: 

27/27 [==============================] - 0s 394us/step - loss: 0.0068 - accuracy: 1.0000
Epoch 161/200
27/27 [==============================] - 0s 467us/step - loss: 0.0155 - accuracy: 1.0000
Epoch 162/200
27/27 [==============================] - 0s 453us/step - loss: 0.0033 - accuracy: 1.0000
Epoch 163/200
27/27 [==============================] - 0s 436us/step - loss: 0.0184 - accuracy: 1.0000
Epoch 164/200
27/27 [==============================] - 0s 530us/step - loss: 0.0116 - accuracy: 1.0000
Epoch 165/200
27/27 [==============================] - 0s 805us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 166/200
27/27 [==============================] - 0s 455us/step - loss: 0.0189 - accuracy: 1.0000
Epoch 167/200
27/27 [==============================] - 0s 397us/step - loss: 0.0289 - accuracy: 1.0000
Epoch 168/200
27/27 [==============================] - 0s 403us/step - loss: 0.0039 - accuracy: 1.0000
Epoch 169/200
27/27 [==============================] - 0s 439us/step - loss: 0.0032 - a